<a href="https://colab.research.google.com/github/Kabongosalomon/Colab-SSH-and-GDrive/blob/master/CUDA_GPU_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CUDA_GPU_server

Make surre to use : GPU runtime mode (Runtime->Change Runtime type -> python3 + GPU
)



In [1]:
# Check nvidia and nvcc cuda compiler

!nvidia-smi
!/usr/local/cuda/bin/nvcc --version

Wed Jul 15 21:09:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Mount Goolge Drive

In [2]:
# link to google drive, click on the given link and choose the google drive account you would like to be available to you 

from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [4]:
#check that Gdrive is mounted

!ls '/content/gdrive/'

'My Drive'  'Shared drives'


#Setup SSH port forwarding

In [5]:
#1 - setup ssh/user 


#Generate a random root password
import random, string
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(30))


#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config

print("username: root")
print("password: ", password)

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')


Creating config file /etc/ssh/sshd_config with new version
Creating SSH2 RSA key; this may take some time ...
2048 SHA256:YVSlxc1QnF4Yi7IzmvNwQK5Ad7ECN9lrzNDGmnQkWFQ root@bc538533c643 (RSA)
Creating SSH2 ECDSA key; this may take some time ...
256 SHA256:EI20R12QfGyd9DfUmqliC3nCP6DQ+e3PNddkLjokOgM root@bc538533c643 (ECDSA)
Creating SSH2 ED25519 key; this may take some time ...
256 SHA256:OO23h4aI6u1jQszJnLgGozzUs6XmZjvfaJpYXhCKKXE root@bc538533c643 (ED25519)
Created symlink /etc/systemd/system/sshd.service → /lib/systemd/system/ssh.service.
Created symlink /etc/systemd/system/multi-user.target.wants/ssh.service → /lib/systemd/system/ssh.service.
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
username: root
password:  ihcIIIUhb6yK4nfiXX8V5EMZqSwnQc


In [6]:
# 2 - Download Ngrok

! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip

In [7]:
# 3 - setup Ngrok - authtoken

#Ask token
print("Get your authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &') # 1bVLy1kHkmWRqmhUEeEoDrUKech_6sRTcUMqizbwwFXt9mtMS 

Get your authtoken from https://dashboard.ngrok.com/auth
··········


**Congratulations, you are ready to go.** 

Now : Go on Ngrok interface https://dashboard.ngrok.com/status you'll find the tcp address and the port 

connect using the following : 

```
ssh root@[tcp address] -p [ngrok_port]

> then enter the password generated previously

```

Example 

```
ssh root@2.tcp.ngrok.io -p 15472
```

# Code to run for installing some usefull tools

```
sudo apt-get install tmux && pip3 install jupyterlab && tmux


jupyter lab --ip 0.0.0.0 --no-browser --allow-root

ctrl+b + shift+"

ssh-keygen -t rsa -b 4096 -C "salomon@aims.ac.za"

ssh -R localhost:80:localhost:8888 jupyterlab@ssh.localhost.run
```

In [ ]:
# jupyter lab --ip 0.0.0.0 --port 56784

# ssh -R 80:localhost:56784 jupyterlab@ssh.localhost.run

# ssh-keygen -t rsa -b 4096 -C "your_email@example.com"

You can optionally use the following code to get some details information about your GPU. 

```
# make nvidia run on colab
export LD_PRELOAD=/usr/lib64-nvidia/libnvidia-ml.so
watch -n 1 nvidia-smi # GPU info 
```

and the following if you're planning running some git commands. 

```
git config --global user.email "salomon@aims.ac.za"
git config --global user.name "Colab Jupyterlab"
```

<font color='red'>Lastly and most importantly, put the follwoing code in the console of the develloper mode(`Ctrl+Shift+i`) of your navigator to keep the Colab session opening as much as possible</font>

```
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
```

<font color='red'>Please remember to run the cell below when you're done. This is neccesary to kill the Ngrok tunel</font>

In [ ]:
# When done, kill Ngrok

!kill $(ps aux | grep './ngrok' | awk '{print $2}')

^C


<h1>About the Author:</h1> 

<a href="https://skabongo.github.io/">Salomon Kabongo</a>, Master degree student at <a href="https://aimsammi.org/">the African Master in Machine Intelligence (AMMI, Ghana)</a> his research focused on the use machine learning technique in the field of Natural Language Processing, learn more about him [here](https://skabongo.github.io/) or [twitter](https://twitter.com/SalomonKabongo1).

**References :** 
- https://imadelhanafi.com/posts/google_colal_server/<br>
- https://medium.com/@archie9211/guide-to-connect-to-google-colab-with-ssh-from-terminal-and-run-jupyter-lab-7ed60258368 <br>
- https://github.com/vdivakar/Colab-SSH-and-GDrive

Copyright &copy; 2020. This notebook and its source code are released under the terms of the <a href="https://www.apache.org/licenses/LICENSE-2.0">Apache License 2.0</a>.